In [1]:
#SPY ETFs buy and hold
import pandas as pd
import numpy as np
from empyrical import max_drawdown
import matplotlib.pyplot as plt

def yearly_returns(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True) 
    returns = {}
    for year in range(1999, 2025): 
        start_date = f"{year-1}-12-31"
        end_date = f"{year}-12-31"
        start_price = df['Adj Close'].loc[:start_date].iloc[-1]
        end_price = df['Adj Close'].loc[:end_date].iloc[-1]
        yearly_return = (end_price - start_price) / start_price
        returns[year] = yearly_return
    return pd.Series(returns)
etfs=['xlb', 'xle', 'xlf', 'xli', 'xlk', 'xlp', 'xlu', 'xlv', 'xly']
for etf in etfs:
    data = {
        'etf': pd.read_csv(f"{etf}_weekly_return_detailed.csv"),
    }
    
    yearly_returns_dict = {key: yearly_returns(df) for key, df in data.items()}
    final_df = pd.DataFrame(yearly_returns_dict)
    final_df.index.name = 'Date'
    
    cash_annual_bh = []
    cash_bh = 100
    for year in range(2000, 2025):
        total_returns_bh = cash_bh * (1 + final_df.loc[year]).sum()
        cash_bh=total_returns_bh
        cash_annual_bh.append((year, total_returns_bh))
    
    years_w, cash_values_bh = zip(*cash_annual_bh)
    data=cash_annual_bh
    years = [item[0] for item in data]
    values = [item[1] for item in data]
    
    df1 = pd.DataFrame({'Year': years, 'Value': values})
    initial_value = 100
    df1['Annualized_Return'] = (df1['Value'] - df1['Value'].shift(1, fill_value=initial_value)) / df1['Value'].shift(1, fill_value=initial_value)*100
    
    
    df=pd.read_csv(f"{etf}_weekly_return_detailed.csv")
    df=df[df["Year"]>=2000]
    
    yearly_volatility = df.groupby('Year')['Return'].std() * np.sqrt(252)
    
    df1 = df1.merge(yearly_volatility.rename('Yearly_Volatility'), on='Year', how='left')
    
    df1["Sharpe_Ratio"]=df1["Annualized_Return"]/df1["Yearly_Volatility"]
    df1.set_index("Year",inplace=True)
    df["Return"]=df["Return"]/100
    yearly_max_drawdown = df.groupby("Year")["Return"].apply(lambda x: max_drawdown(x))
    df1["Max_drawdown"]=yearly_max_drawdown
    df1["Max_drawdown"]=df1["Max_drawdown"]*100
    df0=df1
    df0.to_csv(f"{etf}_bh.csv")